# TCC Forense AI - Playground Interativo

Esse notebook serve como um **painel de controle visual** para o RAG. 
Diferente do script `main.py` (que é focado em execução/produção), aqui é possível:

1. **Inspecionar dados** passo a passo.
2. **Visualizar o progresso** da ingestão.
3. **Debugar** o que o ChromaDB está recuperando (Evidências vs. Leis).
4. **Testar o LLM** rapidamente.

### 1. Configuração do Ambiente
Carrega as variáveis de ambiente e garante que o Python esteja "enxergando" a pasta `src` corretamente.
Isso evita erros de importação se você estiver rodando o Jupyter de uma subpasta

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path

BASE_DIR = Path(os.getcwd())
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

from src.config import DATA_DIR, MODELS_DIR, VECTOR_STORE_DIR

print(f"Diretório Base: {BASE_DIR}")
print(f"Banco Vetorial: {VECTOR_STORE_DIR}")
print(f"Modelos: {MODELS_DIR}")

### 2. Ingestão de Dados (Visual)
Aqui testamos a função de ingestão com um limite pequeno (`limit_files=50`) para validar se os arquivos estão sendo lidos corretamente sem precisar processar o dataset inteiro

In [ ]:
from src.ingestion import ingest_evidence, ingest_legal

print("Testando Ingestão de Jurisprudência (Amostra)...")
ingest_legal(limit_docs=10) # Baixa apenas 10 decisões

print("\nTestando Ingestão de E-mails Enron (Amostra)...")
ingest_evidence(limit_files=50) # Lê apenas 50 arquivos

### 3. Diagnóstico de Retrieval
Antes de ligar a IA (Llama 3), precisamos saber se o **ChromaDB** está encontrando as provas certas.
A função `run_diagnostics` mostra exatamente quais documentos o sistema recupera para uma pergunta

In [ ]:
from src.rag_engine import run_diagnostics

PERGUNTA_TESTE = "Existem reuniões suspeitas sobre preços de gás?"

run_diagnostics(PERGUNTA_TESTE)

### 4. Chat com o Modelo (RAG Completo)
Carrega o modelo Llama 3 e gera a resposta final baseada nas evidências recuperadas acima

In [ ]:
from src.rag_engine import get_rag_chain

print("Carregando Modelo Llama 3...")
chain = get_rag_chain()

question = "Quem é Bill Williams e qual o envolvimento dele com a manipulação de mercado? Cite as fontes."

print(f"\nPergunta: {question}\n")
print("-" * 50)
response = chain.invoke({"question": question})
print("\n" + "-" * 50)